

---


# **Twitter Data Collection**
#### ***Look before you Leap : Leveraging Predictive Models to Improve Automotive Safety and Travel Time***



---



**Final Project**
CSCI 35300/79502   |   Fall 2020 

##### Hannah Do








#### Used Snscrape and Tweepy to collect queried tweets :

In [1]:
! pip3 install snscrape

In [2]:
! pip install tweepy

In [103]:
# Storing twitter URLs in according folders

## TotalTrafficNYC data inconsistent  - sparse at the beginning of the year
## NYPD traffic and NY_DOT - too much irrelevant information / news or ad centered tweets

# ! snscrape twitter-search "@511NY since:2020-06-01 until:2020-06-15" > data_twitter/511_6a.txt
# ! snscrape twitter-search "@511NY since:2020-06-16 until:2020-07-01" > data_twitter/511_6b.txt

# ! snscrape twitter-search "@511NY since:2020-07-02 until:2020-07-15" > data_twitter/511_7a.txt
# ! snscrape twitter-search "@511NY since:2020-07-16 until:2020-08-01" > data_twitter/511_7b.txt

# ! snscrape twitter-search "@511NY since:2020-08-02 until:2020-08-15" > data_twitter/511_8a.txt
! snscrape twitter-search "@511NY since:2020-08-16 until:2020-09-01" > data_twitter/511_8b.txt

! snscrape twitter-search "@511NY since:2020-09-02 until:2020-09-15" > data_twitter/511_9a.txt
# ! snscrape twitter-search "@511NY since:2020-09-16 until:2020-10-01" > data_twitter/511_9b.txt

# ! snscrape twitter-search "@511NY since:2020-10-02 until:2020-10-15" > data_twitter/511_10a.txt
# ! snscrape twitter-search "@511NY since:2020-10-16 until:2020-11-01" > data_twitter/511_10b.txt

# ! snscrape twitter-search "@511NY since:2020-11-02 until:2020-11-15" > data_twitter/511_11a.txt
# ! snscrape twitter-search "@511NY since:2020-11-16 until:2020-12-01" > data_twitter/511_11b.txt



In [104]:

# Import all the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

import datetime, string, re
import sys

import tweepy
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from collections import Counter


sid = SentimentIntensityAnalyzer()
# initializing Vader Sentiment Analyzer


print('All Libraries Sucessfully Imported')


All Libraries Sucessfully Imported


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/doguma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/doguma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/doguma/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


---
#### Twitter Developer Accounts required for API keys :

In [105]:

consumer_key = 'bOgKgIHGuCybRIOFxdxTuFWM3'
consumer_secret = 'XcP6KypHOQurNQvySOVtqy4Fz6TyvOHNmFwAmMmF54AKTG88Ui'
access_token = '1317375977203265537-305NGnHrsADD2EqTqbZGRU9ADAmMFx'
access_token_secret = 'dypXTOkWumb4mC8mtZkrHN4s2q8agkuh8BORi41zLBhk8'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [106]:
def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": status.id,
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at}
#             tweet_elem = {
#                      "tweet":status.full_text,
#                 "date":status.created_at
#                         }
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    return empty_data

- Timeline segmented to smaller units due to long processing hours :

In [107]:

# '511_6a', '511_6b', '511_7a', '511_7b', '511_8a', '511_8b', '511_9a', '511_9b', '511_10a', '511_10b', '511_11a'
t_files = ['511_6a']


-  txt files (link of the tweets) converted to csv files (content of the tweets) :

In [109]:

for t in t_files:

    tweet_url = pd.read_csv("data_twitter/" + t +".txt", index_col= None, header = None, names = ["links"])
    tweet_url.head()

    af = lambda x: x["links"].split("/")[-1]
    tweet_url['id'] = tweet_url.apply(af, axis=1)

    ids = tweet_url['id'].tolist()

    total_count = len(ids)
    chunks = (total_count - 1) // 50 + 1


    for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        temp2 = pd.DataFrame(fetch_tw(batch))
        temp2.to_csv("data_twitter/" + t +".csv", mode="a")


- each csv file (content of the tweets) read as dataframe :

- ***start from here for geocoding process***

In [95]:
# 511_6a.csv, 511_6b.csv, 511_7a.csv, 511_7b.csv,
# 511_8a.csv, 511_8b.csv, 511_9a.csv, 511_9b.csv,
# 511_10a.csv, 511_10b.csv, 511_11a.csv, 511_11b.csv

# processed without a loop since each file takes a long time

tweet_text = pd.read_csv("data_twitter/511_6a.csv", index_col= None, header = None)
tweet_text

temp = pd.DataFrame()
temp['tweet'] = tweet_text[3]

temp.drop(temp.head(1).index,inplace=True)

temp['score_neutral'] = temp['tweet'].apply(lambda x: list(sid.polarity_scores(str(x)).values())[1])
# get neutral sentiment of the tweet

temp['score_polar'] = temp['tweet'].apply(lambda x: list(sid.polarity_scores(str(x)).values())[3])
# get compound score - overall polar sentiment of the tweet

temp


,tweet,score_neutral,score_polar
1,Incident on #NorthLine at New York Penn Statio...,1.000,0.0000
2,Special Event on #MeadowlandsRacetrack https:/...,0.597,0.4019
3,Closure on #NewJerseyTurnpikeEasternSpur NB at...,1.000,0.0000
4,Cleared: Construction on #NY19 NB from West Av...,0.877,0.1027
5,Incident on #I84 WB at 0.12 miles beyond Exit ...,1.000,0.0000
...,...,...,...
31451,Construction on #US19TonnelleAvenue NB from No...,1.000,0.0000
31452,Construction on #NJ38 EB from I-295 to US 206 ...,1.000,0.0000
31453,Construction on #NJ73 SB from CR 534/Jackson R...,1.000,0.0000
31454,Cleared: Special Event on #VerrazanonarrowsBri...,0.709,0.4767


- text processing

In [96]:
# separate camel cases and remove punctuations
# not needed for geocoding

def separate_camelcase(text):
    text = re.sub(r'\'s', '',text)
    text = re.sub(r'\'', '',text)
    text = re.sub(r'[^\w\s]',' ',text)
    text = re.sub(r"([a-z])([A-Z]+)", r"\1 \2", str(text))

    return text


# Replace acronyms and remove redundant words

def replace_acronyms(text):
    # text  = "".join([char for char in text if char not in excluded_words])  
    text = re.sub('i76', ' interstate 76 ', text)
    text = re.sub('i84', ' interstate 84 ', text)
    text = re.sub('i87', ' interstate 87 ', text)
    text = re.sub('i91', ' interstate 91 ', text)
    text = re.sub('i95', ' interstate 95 ', text)
    text = re.sub('i190', ' interstate 190 ', text)
    text = re.sub('i295', ' interstate 295 ', text)
    text = re.sub('i878', ' interstate 878 ', text)
    text = re.sub('us 9', ' U.S. route 9 ', text)
    text = re.sub('us9', ' U.S. route 9 ', text)
    text = re.sub('us 130', ' U.S. route 130 ', text)
    text = re.sub('us130', ' U.S. route 130 ', text)
    
    text = re.sub('eb', ' east bound ', text)
    text = re.sub('sb', ' south bound ', text)
    text = re.sub('nb', ' north bound ', text)
    text = re.sub('wb', ' west bound ', text)

    text = re.sub('queens', ' queens ', text)
    text = re.sub('nj', ' new jersey ', text)
    text = re.sub('nyc', ' new york city ', text)
    text = re.sub('brooklyn', ' brooklyn ', text)
    text = re.sub('new york', ' new york ', text)        
    text = re.sub('staten island', ' staten island ', text)
    text = re.sub('newyork', ' new york ', text)
    text = re.sub('nypd', '', text)
    text = re.sub('ny', ' new york ', text)
    text = re.sub('bronx', ' bronx ', text)
#     text = re.sub('accident', '', text)
#     text = re.sub('parks', '', text)
#     text = re.sub('sirens', '', text)
#     text = re.sub('incident', '', text)
    text = re.sub('rte', ' route ', text)
    text = re.sub('rt', ' route ', text)
    text = re.sub('bklyn', ' brooklyn ', text)
    text = re.sub('bk', ' brooklyn ', text)
    text = re.sub('way', ' way ', text)
    text = re.sub('expy', ' expressway ', text)
#     text = re.sub('crash', '', text)
    text = re.sub('aka', '', text)
    text = re.sub('bqe', ' brooklyn queens expressway ', text)
    text = re.sub('511ny', '', text)

    return text 

temp['separate camel cases'] = temp['tweet'].apply(lambda x: separate_camelcase(x))

temp['replace acronyms'] = temp['separate camel cases'].apply(lambda x: x.lower())

temp['replace acronyms'] = temp['replace acronyms'].apply(lambda x: replace_acronyms(x))

temp

,tweet,score_neutral,score_polar,separate camel cases,replace acronyms
1,Incident on #NorthLine at New York Penn Statio...,1.000,0.0000,Incident on North Line at New York Penn Stati...,incident on no route h line at new york pen...
2,Special Event on #MeadowlandsRacetrack https:/...,0.597,0.4019,Special Event on Meadowlands Racetrack https ...,special event on meadowlands racetrack https ...
3,Closure on #NewJerseyTurnpikeEasternSpur NB at...,1.000,0.0000,Closure on New Jersey Turnpike Eastern Spur N...,closure on new jersey turnpike eastern spur ...
4,Cleared: Construction on #NY19 NB from West Av...,0.877,0.1027,Cleared Construction on NY19 NB from West Av...,cleared construction on new york 19 no rou...
5,Incident on #I84 WB at 0.12 miles beyond Exit ...,1.000,0.0000,Incident on I84 WB at 0 12 miles beyond Exit ...,incident on interstate 84 west bound at 0...
...,...,...,...,...,...
31451,Construction on #US19TonnelleAvenue NB from No...,1.000,0.0000,Construction on US19Tonnelle Avenue NB from N...,construction on us19tonnelle avenue no route...
31452,Construction on #NJ38 EB from I-295 to US 206 ...,1.000,0.0000,Construction on NJ38 EB from I 295 to US 206 ...,construction on new jersey 38 east bound f...
31453,Construction on #NJ73 SB from CR 534/Jackson R...,1.000,0.0000,Construction on NJ73 SB from CR 534 Jackson R...,construction on new jersey 73 south bound ...
31454,Cleared: Special Event on #VerrazanonarrowsBri...,0.709,0.4767,Cleared Special Event on Verrazanonarrows Br...,cleared special event on verrazanonarrows br...


In [97]:
stopword = nltk.corpus.stopwords.words('english')

In [99]:

# Tokenize the text
def tokenization(text):
    text = re.split('\W+', text)
    return text

temp['remove stopwords'] = temp['replace acronyms'].apply(lambda x: tokenization(x))


# Remove stopwords

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
temp['remove stopwords'] = temp['remove stopwords'].apply(lambda x: remove_stopwords(x))

temp.head(10)

# Lemmatization and Stemming skipped due to losing of necessary word forms

# Convert the tokens back to string

def listToString(s):  
    str1 = ""  
      
    for ele in s:  
        str1 += ele + " "  
      
    return str1

temp['remove stopwords'] = temp['remove stopwords'].apply(lambda x: listToString(x))
temp


,tweet,score_neutral,score_polar,separate camel cases,replace acronyms,remove stopwords
1,Incident on #NorthLine at New York Penn Statio...,1.000,0.0000,Incident on North Line at New York Penn Stati...,incident on no route h line at new york pen...,incident route h line new york penn station ht...
2,Special Event on #MeadowlandsRacetrack https:/...,0.597,0.4019,Special Event on Meadowlands Racetrack https ...,special event on meadowlands racetrack https ...,special event meadowlands racetrack https co r...
3,Closure on #NewJerseyTurnpikeEasternSpur NB at...,1.000,0.0000,Closure on New Jersey Turnpike Eastern Spur N...,closure on new jersey turnpike eastern spur ...,closure new jersey turnpike eastern spur route...
4,Cleared: Construction on #NY19 NB from West Av...,0.877,0.1027,Cleared Construction on NY19 NB from West Av...,cleared construction on new york 19 no rou...,cleared construction new york 19 route h bound...
5,Incident on #I84 WB at 0.12 miles beyond Exit ...,1.000,0.0000,Incident on I84 WB at 0 12 miles beyond Exit ...,incident on interstate 84 west bound at 0...,incident interstate 84 west bound 0 12 miles b...
...,...,...,...,...,...,...
31451,Construction on #US19TonnelleAvenue NB from No...,1.000,0.0000,Construction on US19Tonnelle Avenue NB from N...,construction on us19tonnelle avenue no route...,construction us19tonnelle avenue route h bound...
31452,Construction on #NJ38 EB from I-295 to US 206 ...,1.000,0.0000,Construction on NJ38 EB from I 295 to US 206 ...,construction on new jersey 38 east bound f...,construction new jersey 38 east bound 295 us 2...
31453,Construction on #NJ73 SB from CR 534/Jackson R...,1.000,0.0000,Construction on NJ73 SB from CR 534 Jackson R...,construction on new jersey 73 south bound ...,construction new jersey 73 south bound cr 534 ...
31454,Cleared: Special Event on #VerrazanonarrowsBri...,0.709,0.4767,Cleared Special Event on Verrazanonarrows Br...,cleared special event on verrazanonarrows br...,cleared special event verrazanonarrows bridge ...


#### Geocoding on each text through ArcGIS : returns location (longitude, latitude) of the text
- takes at least 5 hours to process

In [ ]:

from arcgis.gis import GIS
from arcgis.geocoding import geocode
my_gis = GIS()

temp["longitude"] = pd.to_numeric(temp2["longitude"], downcast="float")
temp["latitude"] = pd.to_numeric(temp2["latitude"], downcast="float")

In [ ]:

temp2 = pd.DataFrame()
temp2 = temp

temp2 = temp.iloc[10]


In [110]:
# adding geocode on each string, if no geocode retrieved - return (0,0)

i=1

while i <len(temp2['remove stopwords'])+1:
    string_temp = str(temp2['remove stopwords'][i])
    string_temp = string_temp + ' new york'
    
    if len(string_temp) < 100:    # maximum string length limit for arcgis geocoding
        locat = geocode(string_temp)
        if (locat):
            temp2['longitude'][i] = locat[0]['location']['x']
            temp2['latitude'][i] = locat[0]['location']['y']
        else:
            temp2['latitude'][i] = 0
            temp2['longitude'][i] = 0            
    else:
        temp2['latitude'][i] = 0
        temp2['longitude'][i] = 0
    i += 1
        

In [111]:
temp2['longitude'] = temp2['longitude'].apply(lambda x: float(x))
temp2['latitude'] = temp2['latitude'].apply(lambda x: float(x))

temp2 = temp2[temp2['latitude'] > 40.5]
temp2 = temp2[temp2['latitude'] < 41]
temp2 = temp2[temp2['longitude'] < -73]
temp2 = temp2[temp2['longitude'] > -75]

temp2.to_csv('data_twitter/locat_6a.csv', mode='a')


In [34]:
temp2 = temp2[temp2['latitude'] !=0]
temp2.drop(columns = ['separate camel cases', 'replace acronyms'], inplace=True)
temp2

,tweet,score_neutral,score_polar,remove stopwords,latitude,longitude
1,Cleared: Incident on #B67Bus at Brooklyn,0.781,0.1027,cleared incident b67bus brooklyn,40.692450,-73.990360
2,Cleared: Incident on #B48Bus at Brooklyn,0.781,0.1027,cleared incident b48bus brooklyn,40.678507,-73.944169
3,Cleared: Incident on #B83Bus at Brooklyn,0.781,0.1027,cleared incident b83bus brooklyn,40.692450,-73.990360
4,Cleared: Incident on #B4Bus Both directions at...,0.833,0.1027,cleared incident b4bus directions brooklyn,40.692450,-73.990360
5,Closure on #US19TonnelleAvenue SB at Secaucus ...,1.000,0.0000,closure us19tonnelle avenue south bound secauc...,40.714550,-74.007140
...,...,...,...,...,...,...
8864,Construction on #NY43 Both directions from Mea...,1.000,0.0000,construction new york 43 directions meadow lan...,40.555151,-74.142154
8871,Construction on #I787 NB at Exit 6; NY 32; War...,1.000,0.0000,construction i787 route h bound exit 6 new yor...,40.714550,-74.007140
8872,Construction on #NY7 EB at Exit to US 9 https:...,1.000,0.0000,construction new york 7 east bound exit us 9 h...,40.714550,-74.007140
8873,Construction on #I84 EB at Exit 46 (WEST BOULE...,1.000,0.0000,construction interstate 84 east bound exit 46 ...,40.714550,-74.007140


In [35]:
temp2.columns = ['original tweet', 'neutral score', 'polar score', 'trimmed tweet', 'latitude', 'longitude']
temp2

,original tweet,neutral score,polar score,trimmed tweet,latitude,longitude
1,Cleared: Incident on #B67Bus at Brooklyn,0.781,0.1027,cleared incident b67bus brooklyn,40.692450,-73.990360
2,Cleared: Incident on #B48Bus at Brooklyn,0.781,0.1027,cleared incident b48bus brooklyn,40.678507,-73.944169
3,Cleared: Incident on #B83Bus at Brooklyn,0.781,0.1027,cleared incident b83bus brooklyn,40.692450,-73.990360
4,Cleared: Incident on #B4Bus Both directions at...,0.833,0.1027,cleared incident b4bus directions brooklyn,40.692450,-73.990360
5,Closure on #US19TonnelleAvenue SB at Secaucus ...,1.000,0.0000,closure us19tonnelle avenue south bound secauc...,40.714550,-74.007140
...,...,...,...,...,...,...
8864,Construction on #NY43 Both directions from Mea...,1.000,0.0000,construction new york 43 directions meadow lan...,40.555151,-74.142154
8871,Construction on #I787 NB at Exit 6; NY 32; War...,1.000,0.0000,construction i787 route h bound exit 6 new yor...,40.714550,-74.007140
8872,Construction on #NY7 EB at Exit to US 9 https:...,1.000,0.0000,construction new york 7 east bound exit us 9 h...,40.714550,-74.007140
8873,Construction on #I84 EB at Exit 46 (WEST BOULE...,1.000,0.0000,construction interstate 84 east bound exit 46 ...,40.714550,-74.007140


In [37]:
temp2 = temp2.reset_index(drop=True)

temp2.to_csv('data_twitter/locat_6a.csv', mode='a')

In [38]:

freq = pd.DataFrame()
freq2 = temp2.groupby(["longitude", "latitude"]).size().reset_index(name="Frequency")

freq2 = freq2.sort_values('Frequency', ascending=False)

freq2.drop(freq2.head(1).index,inplace=True)
# most frequent value is the default value that returns location of 'new york'

freq2[['polar score', 'neutral score']] = temp2[['polar score', 'neutral score']]

freq2

,longitude,latitude,Frequency,polar score,neutral score
175,-73.990360,40.692450,84,0.0000,1.000
440,-73.829990,40.714000,46,0.0000,1.000
301,-73.946793,40.680002,44,0.0000,1.000
72,-74.080140,40.644550,39,0.1027,0.833
320,-73.942942,40.849800,33,0.0000,1.000
...,...,...,...,...,...
375,-73.911000,40.876410,1,0.1027,0.903
376,-73.909710,40.860190,1,0.1027,0.903
377,-73.909344,40.865739,1,0.1027,0.903
378,-73.906270,40.849390,1,0.0000,1.000
